In [ ]:
# "Image Classification Using CIFAR-10 Dataset using simple deep network with 4 hidden layers and 3 dropout layer also apply pruning and quantization
to reduce size and report size of model"

In [ ]:
!pip install tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot  # For pruning and quantization
import numpy as np
import os

# -------------------------------
# Load and Preprocess CIFAR-10 Dataset
# -------------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values (0 to 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# -------------------------------
#  Build a Simple Deep Neural Network (DNN)
# -------------------------------
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # 10 classes (CIFAR-10)
    ])
    return model

# Create and compile the model
original_model = create_model()
original_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# -------------------------------
# Train the Original Model
# -------------------------------
original_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), batch_size=64)

# Save original model and get size
original_model.save('original_model.h5')
original_model_size = os.path.getsize('original_model.h5') / (1024 * 1024)  # Convert to MB
print(f"Original Model Size: {original_model_size:.2f} MB")

# -------------------------------
# Apply Pruning to Reduce Parameters
# -------------------------------
pruning_params = {
    "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20, final_sparsity=0.80, begin_step=0, end_step=1000
    )
}

# Apply pruning wrapper
pruned_model = tf.keras.Sequential([
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Flatten(input_shape=(32, 32, 3)), **pruning_params),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(512, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(256, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(128, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(64, activation='relu'), **pruning_params),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(10, activation='softmax'), **pruning_params)
])

# Compile pruned model
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the pruned model with pruning callback
pruned_model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[tfmot.sparsity.keras.UpdatePruningStep()]
)




# Remove pruning wrappers before saving
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)
stripped_pruned_model.save('pruned_model.h5')

# Get pruned model size
pruned_model_size = os.path.getsize('pruned_model.h5') / (1024 * 1024)
print(f"Pruned Model Size: {pruned_model_size:.2f} MB")

# -------------------------------
# Apply Quantization for Further Compression
# -------------------------------
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply full quantization

quantized_model = converter.convert()

# Save quantized model
quantized_model_path = 'quantized_model.tflite'
with open(quantized_model_path, 'wb') as f:
    f.write(quantized_model)

# Get quantized model size
quantized_model_size = os.path.getsize(quantized_model_path) / (1024 * 1024)
print(f"Quantized Model Size: {quantized_model_size:.2f} MB")

# -------------------------------
# Compare Model Sizes
# -----------------------------
print("\nModel Size Comparison:")
print(f"Original Model: {original_model_size:.2f} MB")
print(f"Pruned Model (After Stripping): {pruned_model_size:.2f} MB")
print(f"Quantized Model: {quantized_model_size:.2f} MB")

Epoch 1/10
782/782 [==============================] - 32s 39ms/step - loss: 2.0457 - accuracy: 0.2373 - val_loss: 1.8939 - val_accuracy: 0.3063
Epoch 2/10
782/782 [==============================] - 30s 38ms/step - loss: 1.8857 - accuracy: 0.3052 - val_loss: 1.8209 - val_accuracy: 0.3581
Epoch 3/10
782/782 [==============================] - 30s 38ms/step - loss: 1.8362 - accuracy: 0.3281 - val_loss: 1.7524 - val_accuracy: 0.3769
Epoch 4/10
782/782 [==============================] - 30s 38ms/step - loss: 1.8113 - accuracy: 0.3356 - val_loss: 1.7117 - val_accuracy: 0.3881
Epoch 5/10
782/782 [==============================] - 30s 39ms/step - loss: 1.7911 - accuracy: 0.3478 - val_loss: 1.7169 - val_accuracy: 0.4004
Epoch 6/10
782/782 [==============================] - 31s 39ms/step - loss: 1.7628 - accuracy: 0.3581 - val_loss: 1.6632 - val_accuracy: 0.4173
Epoch 7/10
782/782 [==============================] - 29s 37ms/step - loss: 1.7465 - accuracy: 0.3653 - val_loss: 1.6448 - val_accuracy:

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Original Model Size: 20.04 MB
Epoch 1/10
782/782 [==============================] - 39s 44ms/step - loss: 2.0200 - accuracy: 0.2512 - val_loss: 1.8009 - val_accuracy: 0.3508
Epoch 2/10
782/782 [==============================] - 32s 40ms/step - loss: 1.8284 - accuracy: 0.3353 - val_loss: 1.7247 - val_accuracy: 0.3806
Epoch 3/10
782/782 [==============================] - 32s 41ms/step - loss: 1.7725 - accuracy: 0.3576 - val_loss: 1.6699 - val_accuracy: 0.3980
Epoch 4/10
782/782 [==============================] - 32s 40ms/step - loss: 1.7361 - accuracy: 0.3748 - val_loss: 1.6320 - val_accuracy: 0.4163
Epoch 5/10
782/782 [==============================] - 35s 44ms/step - loss: 1.7018 - accuracy: 0.3845 - val_loss: 1.6031 - val_accuracy: 0.4301
Epoch 6/10
782/782 [==============================] - 32s 41ms/step - loss: 1.6715 - accuracy: 0.3952 - val_loss: 1.6004 - val_accuracy: 0.4300
Epoch 7/10
782/782 [==============================] - 32s 41ms/step - loss: 1.6480 - accuracy: 0.4057 - va

Pruned Model Size: 6.69 MB
Quantized Model Size: 1.68 MB

Model Size Comparison:
Original Model: 20.04 MB
Pruned Model (After Stripping): 6.69 MB
Quantized Model: 1.68 MB
